In [3]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [49]:
cfg = Config.fromfile('/opt/ml/Swin-Transformer-Object-Detection/configs/swin/cascade_mask_rcnn_swin_small_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='CascadeRCNN',
    pretrained=None,
    backbone=dict(
        type='SwinTransformer',
        embed_dim=96,
        depths=[2, 2, 18, 2],
        num_heads=[3, 6, 12, 24],
        window_size=7,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.2,
        ape=False,
        patch_norm=True,
        out_indices=(0, 1, 2, 3),
        use_checkpoint=False),
    neck=dict(
        type='FPN',
        in_channels=[96, 192, 384, 768],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_

In [34]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_2x_coco.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (1024, 1024)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (1333, 800)

cfg.data.samples_per_gpu = 32
cfg.data.workers_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_2x_batch8_wontrans_s50c65_num3'
cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None



# Multi scale
cfg.data.test.pipeline[1]['img_scale'] = [(1000, 600),(1333, 800),(1666, 1000)]
# cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

# checkpoint path
epoch = 24
# checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(typ

In [50]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
# cfg = Config.fromfile('/opt/ml/Swin-Transformer-Object-Detection/configs/swin/siwn_one.py')
cfg = Config.fromfile('/opt/ml/Swin-Transformer-Object-Detection/configs/swin/new_our_swin.py')
# cfg = Config.fromfile('/opt/ml/Swin-Transformer-Object-Detection/configs/swin/noapex.py')

cfg.work_dir = './work_dirs/swin2_1team'

# PREFIX = '/opt/ml/input/data/'

cfg.data.train.classes = classes
# cfg.data.train.img_prefix = PREFIX
# cfg.data.train.ann_file = PREFIX + 'train_all.json'

PREFIX = '/opt/ml/input/data/'

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 4

cfg.seed=2020
# cfg.gpu_ids = [0]

cfg.lr_config = dict(policy='CosineAnnealing',warmup='linear',warmup_iters=3000,
                    warmup_ratio=0.0001, min_lr_ratio=1e-7)

cfg.load_from = "cascade_mask_rcnn_swin_small_patch4_window7.pth" # 얘는 pretrain 모델 가져오는 경로

cfg.gpu_ids = [0]




# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_all.json'
cfg.data.train.seg_prefix=PREFIX                        ### 요놈 ###


cfg.data.train=dict(
        type='CocoDataset',
        ann_file='/opt/ml/input/data/train_all.json',
        img_prefix='/opt/ml/input/data/',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='InstaBoost',
                action_candidate=('normal', 'horizontal', 'skip'),
                action_prob=(1, 0, 0),
                scale=(0.8, 1.2),
                dx=15,
                dy=15,
                theta=(-1, 1),
                color_prob=0.0,
                hflag=False,
                aug_ratio=0.5),
            dict(
                type='LoadAnnotations',
                with_bbox=True,
                with_mask=True,
                with_seg=True),
            dict(type='RandomFlip', flip_ratio=0.5),
            dict(
                type='AutoAugment',
                policies=[[{
                    'type':
                    'Resize',
                    'img_scale':
                    [(512, 512), (576, 576), (640, 640), (704, 704),
                     (768, 768), (832, 832), (896, 896), (960, 960),
                     (1024, 1024)],
                    'multiscale_mode':
                    'value',
                    'keep_ratio':
                    True
                }],
                          [{
                              'type': 'Resize',
                              'img_scale': [(512, 512), (768, 768),
                                            (1024, 1024)],
                              'multiscale_mode': 'value',
                              'keep_ratio': True
                          }, {
                              'type': 'RandomCrop',
                              'crop_type': 'absolute_range',
                              'crop_size': (512, 512),
                              'allow_negative_crop': True
                          }, {
                              'type':
                              'Resize',
                              'img_scale': [(512, 512), (576, 576), (640, 640),
                                            (704, 704), (768, 768), (832, 832),
                                            (896, 896), (960, 960),
                                            (1024, 1024)],
                              'multiscale_mode':
                              'value',
                              'override':
                              True,
                              'keep_ratio':
                              True
                          }]]),
            dict(type='Pad', size_divisor=32),
            dict(
                type='Albu',
                transforms=[
                    dict(
                        type='Cutout',
                        num_holes=30,
                        max_h_size=30,
                        max_w_size=30,
                        fill_value=[103.53, 116.28, 123.675],
                        p=0.1),
                    dict(
                        type='RandomBrightnessContrast',
                        brightness_limit=[0.1, 0.3],
                        contrast_limit=[0.1, 0.3],
                        p=0.2),
                    dict(
                        type='OneOf',
                        transforms=[
                            dict(
                                type='RGBShift',
                                r_shift_limit=10,
                                g_shift_limit=10,
                                b_shift_limit=10,
                                p=1.0),
                            dict(
                                type='HueSaturationValue',
                                hue_shift_limit=20,
                                sat_shift_limit=30,
                                val_shift_limit=20,
                                p=1.0),
                            dict(type='RandomGamma'),
                            dict(type='CLAHE')
                        ],
                        p=0.1),
                    dict(
                        type='JpegCompression',
                        quality_lower=85,
                        quality_upper=95,
                        p=0.2),
                    dict(type='ChannelShuffle', p=0.1),
                    dict(
                        type='OneOf',
                        transforms=[
                            dict(type='Blur', blur_limit=3, p=1.0),
                            dict(type='MedianBlur', blur_limit=3, p=1.0),
                            dict(type='MotionBlur'),
                            dict(type='GaussNoise'),
                            dict(type='ImageCompression', quality_lower=75)
                        ],
                        p=0.1),
                    dict(type='RandomRotate90', p=0.5)
                ],
                bbox_params=dict(
                    type='BboxParams',
                    format='pascal_voc',
                    label_fields=['gt_labels'],
                    min_visibility=0.0,
                    filter_lost_elements=True),
                keymap=dict(img='image', gt_masks='masks', gt_bboxes='bboxes'),
                update_pad_shape=False,
                skip_img_without_anno=True),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='SegRescale', scale_factor=0.125),
            dict(type='DefaultFormatBundle'),
            dict(
                type='Collect',
                keys=[
                    'img', 'gt_bboxes', 'gt_labels', 'gt_masks',
                    'gt_semantic_seg'
                ])
        ],
        classes=('UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal',
                 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery',
                 'Clothing'),
        seg_prefix='/opt/ml/input/data/')

cfg.checkpoint_config = dict(max_keep_ckpts=2, interval=1)

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# Multi scale
# cfg.data.test.pipeline[1]['img_scale'] = [(1000, 600),(1333, 800),(1666, 1000)]
cfg.data.test.pipeline[1]['img_scale'] = [(320, 320), (480, 480), (640, 640), (800,800), (960,960), (1120,1120)]


# cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

# checkpoint path
epoch = 29
# checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')


# cfg.model.test_cfg=dict(
#         rpn=dict(
#             nms_pre=1000,
#             max_per_img=1000,
#             nms=dict(type='nms', iou_threshold=0.7),
#             min_bbox_size=0),
#         rcnn=dict(
#             score_thr=0.001,
#             nms=dict(type='nms', iou_threshold=0.4),
#             max_per_img=100,
#             mask_thr_binary=0.5))

# cfg.model.test_cfg=dict(
#         rpn=dict(
#             nms_pre=1000,
#             max_per_img=1000,
#             nms=dict(type='nms', iou_threshold=0.7),
#             min_bbox_size=0),
#         rcnn=dict(
#             score_thr=0.000,
#             nms=dict(type='nms', iou_threshold=0.4),
#             max_per_img=100,
#             mask_thr_binary=0.5))


print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='InstaBoost',
        action_candidate=('normal', 'horizontal', 'skip'),
        action_prob=(1, 0, 0),
        scale=(0.8, 1.2),
        dx=15,
        dy=15,
        theta=(-1, 1),
        color_prob=0.0,
        hflag=False,
        aug_ratio=0.5),
    dict(
        type='LoadAnnotations', with_bbox=True, with_mask=True, with_seg=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='AutoAugment',
        policies=[[{
            'type':
            'Resize',
            'img_scale': [(512, 512), (576, 576), (640, 640), (704, 704),
                          (768, 768), (832, 832), (896, 896), (960, 960),
                          (1024, 1024)],
            'multiscale_mode':
            'value',
            'keep_ratio':
      

In [51]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [52]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

Use load_from_local loader


In [53]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 0.7 task/s, elapsed: 1180s, ETA:     0s

In [55]:
mmcv.dump(output, "./work_dirs/swin2_1team/epoch_29.pkl")

In [46]:
len(output[3][0])

11

In [54]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    out = out[0]
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_swin_29.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.004770118 162.08495 73.9816 254.09549 163....,batch_01_vt/0021.jpg
1,0 0.005677956 0.05226949 222.2557 106.20748 35...,batch_01_vt/0028.jpg
2,0 0.015770484 233.219 460.03217 303.2897 511.0...,batch_01_vt/0031.jpg
3,0 0.00587218 298.22028 159.11554 361.67505 225...,batch_01_vt/0032.jpg
4,1 0.42366537 388.9406 265.778 488.71725 343.90...,batch_01_vt/0070.jpg


In [6]:
cfg.work_dir = './work_dirs/faster_rcnn_kfold0'
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])
output = single_gpu_test(model, data_loader, show_score_thr=0.05)
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_1x_fold1_.csv'), index=None)
submission.head()

2021-05-16 01:51:10,175 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-16 01:51:10,176 - mmdet - INFO - Use load_from_torchvision loader
2021-05-16 01:51:10,592 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 8.4 task/s, elapsed: 99s, ETA:     0sloading annotations into memory...
Done (t=0.00s)
creating index...
index created!


,PredictionString,image_id
0,1 0.27244276 156.50906 65.47758 256.4415 166.9...,batch_01_vt/0021.jpg
1,1 0.11728452 102.37038 0.0 196.40446 256.51465...,batch_01_vt/0028.jpg
2,0 0.083333336 106.56012 0.0 336.13547 0.0 0 0....,batch_01_vt/0031.jpg
3,2 0.68128747 296.69302 156.01796 362.49228 233...,batch_01_vt/0032.jpg
4,1 0.26740214 367.3728 166.96545 506.89734 336....,batch_01_vt/0070.jpg


In [7]:
cfg.work_dir = './work_dirs/faster_rcnn_kfold1'
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])
output = single_gpu_test(model, data_loader, show_score_thr=0.05)
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_1x_fold1_.csv'), index=None)
submission.head()

2021-05-16 01:52:51,476 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-16 01:52:51,477 - mmdet - INFO - Use load_from_torchvision loader
2021-05-16 01:52:51,729 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 8.9 task/s, elapsed: 94s, ETA:     0sloading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.06447051 157.11705 55.205334 255.71165 167...,batch_01_vt/0021.jpg
1,1 0.12336707 0.0 216.91666 294.43307 463.7288 ...,batch_01_vt/0028.jpg
2,0 0.083333336 462.4012 0.0 494.1273 0.0 0 0.08...,batch_01_vt/0031.jpg
3,1 0.061352003 336.65472 184.47388 364.66217 23...,batch_01_vt/0032.jpg
4,1 0.16919202 356.94113 161.57474 510.03732 329...,batch_01_vt/0070.jpg


In [8]:
cfg.work_dir = './work_dirs/faster_rcnn_kfold2'
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])
output = single_gpu_test(model, data_loader, show_score_thr=0.05)
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_faster_rcnn_2x_batch8_wontrans_pseudo1_s30c55.csv'), index=None)
submission.head()

2021-05-16 01:54:29,108 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-16 01:54:29,109 - mmdet - INFO - Use load_from_torchvision loader
2021-05-16 01:54:29,328 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 9.3 task/s, elapsed: 90s, ETA:     0sloading annotations into memory...
Done (t=0.00s)
creating index...
index created!


,PredictionString,image_id
0,1 0.2573298 142.61311 49.239693 256.6261 172.7...,batch_01_vt/0021.jpg
1,1 0.20577663 100.95451 0.0 193.90013 251.7756 ...,batch_01_vt/0028.jpg
2,0 0.083333336 297.2498 0.0 505.07144 0.0 0 0.0...,batch_01_vt/0031.jpg
3,1 0.082291365 297.06442 161.45549 364.91428 22...,batch_01_vt/0032.jpg
4,1 0.18720336 153.96759 298.29453 293.4585 407....,batch_01_vt/0070.jpg


In [9]:
cfg.work_dir = './work_dirs/faster_rcnn_kfold3'
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])
output = single_gpu_test(model, data_loader, show_score_thr=0.05)
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_1x_fold3_.csv'), index=None)
submission.head()

2021-05-16 01:56:02,418 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-16 01:56:02,419 - mmdet - INFO - Use load_from_torchvision loader
2021-05-16 01:56:02,652 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 10.2 task/s, elapsed: 82s, ETA:     0sloading annotations into memory...
Done (t=0.00s)
creating index...
index created!


,PredictionString,image_id
0,1 0.36776102 343.9898 35.461994 364.5382 46.19...,batch_01_vt/0021.jpg
1,1 0.16002242 75.58316 78.65596 155.83589 282.6...,batch_01_vt/0028.jpg
2,0 0.083333336 500.0681 0.0 512.0 0.0 0 0.08333...,batch_01_vt/0031.jpg
3,1 0.074704476 295.50296 159.4894 366.369 231.2...,batch_01_vt/0032.jpg
4,1 0.22645006 389.9115 263.38092 494.7198 341.9...,batch_01_vt/0070.jpg


In [10]:
cfg.work_dir = './work_dirs/faster_rcnn_kfold4'
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])
output = single_gpu_test(model, data_loader, show_score_thr=0.05)
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_1x_fold4_.csv'), index=None)
submission.head()

2021-05-16 01:57:27,533 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-16 01:57:27,534 - mmdet - INFO - Use load_from_torchvision loader
2021-05-16 01:57:27,765 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 10.6 task/s, elapsed: 79s, ETA:     0sloading annotations into memory...
Done (t=0.15s)
creating index...
index created!


,PredictionString,image_id
0,1 0.27636898 165.13287 74.08579 254.68384 168....,batch_01_vt/0021.jpg
1,1 0.056884393 276.7744 247.95915 305.57526 262...,batch_01_vt/0028.jpg
2,0 0.083333336 498.7401 0.0 512.0 0.0 0 0.08333...,batch_01_vt/0031.jpg
3,1 0.08946579 295.6789 159.69301 361.1754 231.0...,batch_01_vt/0032.jpg
4,1 0.23603567 156.19955 299.54343 292.8967 400....,batch_01_vt/0070.jpg


In [46]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_2x_1333_batch8.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.7233858 163.10098 66.88278 255.18427 164.8...,batch_01_vt/0021.jpg
1,6 0.12591846 160.01428 154.27768 331.27255 253...,batch_01_vt/0028.jpg
2,1 0.94246507 61.6754 164.9594 388.50378 391.10...,batch_01_vt/0031.jpg
3,0 0.08908267 297.62042 161.78137 362.80948 229...,batch_01_vt/0032.jpg
4,0 0.18186992 39.262924 460.12997 79.42103 473....,batch_01_vt/0070.jpg


In [32]:
lr_goal = 2e-2

In [36]:
lr_goal*((1.0001)**500)

0.021025369367535216

In [35]:
lr_goal

0.02

In [47]:
"1".startswith("111")

False

In [48]:
a = "111"

In [52]:
 "1" not in a

False

In [57]:
def solution(s):
    answer = []
    for i in s:
        start_string = ""   # 우선순위 가장 높은 숫자
        zzo = ""       # 우선순위 두번째 나머지는 i
        while True:
            if i == "10":
                start_string += i
                i = ""
                break
            if i == "110":
                zzo += i
                i = ""
                break
            if i == "1":
                zzo += i
                i = ""
                break
            if "0" not in i:
                break
            if "11" in i:
                index = i.index("11")
                start_string = start_string + i[:index]
                i = i[index:]
            else:
                start_string = start_string + i
                i = ""
                break
            while i.startswith("111"):
                if "0" not in i:
                    break
                zzo += "110"
                index = i.index("110")
                i = i[:index] + i[index+3:]
                print(i)
        answer.append(start_string + zzo + i)
    return answer

In [58]:
solution(["1110","100111100","0111111010"])

1
110
111110
111


['1101', '100110110', '0110110111']

In [59]:
"11" < "110"

True

In [ ]:
def solution(s):
    answer = []
    for i in s:
        one_stack = []
        first_s = ""
        temp = ""
        zzo_cnt = 0
        for j in i:
            if j == "1":
                one_stack.append("1")
            else:
                if len(one_stack) >= 2:
                    one_stack.pop()
                    one_stack.pop()
                    zzo_cnt += 1
                else:
                    first_s += "".join(one_stack) + "0"
                    one_stack = []
        answer.append(first_s + "110"*zzo_cnt + "".join(one_stack))
    return answer